In [ ]:
geo_codes = gc2012["NUTS_ID"].unique().tolist()
print(geo_codes)

In [ ]:
def get_eurostat_animal(animal):
    return eurostat.get_data_df("agr_r_animal", filter_pars= {'animals':animal, 'startPeriod': 2012, 'endPeriod':2018})


In [ ]:
def get_eurostat_pop():
    return eurostat.get_data_df("demo_r_d2jan", filter_pars= {'sex':'T','age':'TOTAL','unit':'NR','startPeriod': 2012, 'endPeriod':2018})


In [ ]:
df_cows = get_eurostat_animal("A2300")
df_cows.drop(["2013","2014","2015","2016","2017"], axis=1, inplace=True)


In [ ]:
df_pop = get_eurostat_pop()
df_pop.drop(["2013","2014","2015","2016","2017"], axis=1, inplace=True)


In [ ]:
mr = gc2012.merge(df_cows, left_on="NUTS_ID", right_on="geo\\TIME_PERIOD", how="left")


In [ ]:
def get_full_data(year, level, primary, secondary, f):
    if primary == "":
        return gc2012
        
    gdf = gc2012
    if year == "2012" and level == "Country":
        gdf = gc2012
    elif year == "2018" and level == "Country":
        gdf = gc2018
    elif year == "2012" and level == "Region":
        gdf = gr2012
    elif year == "2018" and level == "Region":
        gdf = gr2018
    elif year == "2018 to 2012 compare" and level == "Country":
        g18 = gc2018
        g12 = gc2012
        g18[primary] = g18[primary] - g12[primary]
        gdf = g18
    elif year == "2018 to 2012 compare" and level == "Region":
        g18 = gr2018
        g12 = gr2012
        g18[primary] = g18[primary] - g12[primary]
        gdf = g18

    if secondary == "Cows":
        mr = gdf.merge(df_cows, left_on="NUTS_ID", right_on="geo\\TIME_PERIOD", how="left")
        mr["cnt"] = np.where(mr[year] != 0, mr[primary] / mr[year], np.nan)
        gdf = mr
    elif secondary == "Population":
        mr = gdf.merge(df_pop, left_on="NUTS_ID", right_on="geo\\TIME_PERIOD", how="left")
        mr["cnt"] = np.where(mr[year] != 0, mr[primary] / mr[year], np.nan)
        gdf = mr
    else:
        gdf["cnt"] = gdf[primary]

    gdf["p_year"] = year
    gdf["p_level"] = level
    gdf["p_primary"] = primary
    gdf["p_secondary"] = secondary
    gdf["p_f"] = f
    
    return gdf

In [ ]:
zz = get_full_data("2012","Region", "","","")
